In [1]:
import sys
import random
sys.path.append("/pod/2/ke-lab/LUOZ/Singularity/iM6A")

In [2]:
from keras.models import load_model
from pkg_resources import resource_filename
import numpy as np
import pandas as pd
from Bio.Seq import Seq
from Bio.SeqUtils import nt_search
import keras.backend as kb

Using TensorFlow backend.


In [3]:
def one_hot_encode(seq):

    map = np.asarray([[0, 0, 0, 0],
                      [1, 0, 0, 0],
                      [0, 1, 0, 0],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])

    seq = seq.upper().replace('A', '\x01').replace('C', '\x02')
    seq = seq.replace('G', '\x03').replace('T', '\x04').replace('N', '\x00')

    return map[np.fromstring(seq, np.int8) % 5]

In [4]:
def categorical_crossentropy_2d(y_true, y_pred):
    # Standard categorical cross entropy for sequence outputs

    return - kb.mean(y_true[:, :, 0]*kb.log(y_pred[:, :, 0]+1e-10)
                   + y_true[:, :, 1]*kb.log(y_pred[:, :, 1]+1e-10))

In [5]:
context = 10000

In [6]:
paths = ('Test/humanRAC10000_c{}.h5'.format(x) for x in range(1, 6))

In [7]:
models = [load_model(resource_filename('m6AAI', x), custom_objects={'categorical_crossentropy_2d': categorical_crossentropy_2d}) for x in paths]

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


### Read data

In [8]:
Data = pd.read_csv("Fasta_ClinVar_Heatmap.csv")

In [9]:
Data

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,LastExonStart,LastExonEnd,m6AStart,m6AEnd,ProbREF,Sequence,GeneLength,LastExonLength,Pre,PreSequence,LastExonSequence,m6APosition,POS
0,DARS2,chr1,+,173793640,173827684,173794367,173826843,173826655,173827684,173826779,173826780,0.291580,CACTTAACCACTATCGTCAACCTTTCCAATCGTGCTCGGTTGCTCC...,34044,1029,33015,CACTTAACCACTATCGTCAACCTTTCCAATCGTGCTCGGTTGCTCC...,GGTTAGACAGACTGATATGCCTTGTCACTGGATCTCCAAGCATCAG...,124,33139
1,NMNAT1,chr1,+,10003485,10045556,10032131,10042759,10042358,10045556,10042736,10042737,0.100627,GAGAGTGCGACCGAGATGTTCCACTCGCTGGCGTCCGGGCCGCTGG...,42071,3198,38873,GAGAGTGCGACCGAGATGTTCCACTCGCTGGCGTCCGGGCCGCTGG...,CTGTGCCAAAGGTCAAGCTGCTGTGTGGGGCAGATTTATTGGAGTC...,378,39251
2,UBIAD1,chr1,+,11333262,11348491,11333588,11346188,11345700,11348491,11345877,11345878,0.222860,GCGGGCCTCCAGAGGCCGGCGCACAAGATGGCGGCTCTGGCGGCCT...,15229,2791,12438,GCGGGCCTCCAGAGGCCGGCGCACAAGATGGCGGCTCTGGCGGCCT...,GAATTGGATTCAAGTACGTGGCTCTGGGAGACCTCATCATCCTCAT...,177,12615
3,GJB3,chr1,+,35246789,35251965,35250363,35251176,35250338,35251965,35250859,35250860,0.515990,GAACTTCTTTCCTGGCACAGGACTCACTGTGCCCCTTCCCGCTGTG...,5176,1627,3549,GAACTTCTTTCCTGGCACAGGACTCACTGTGCCCCTTCCCGCTGTG...,GTAGGCACGGCCCCCACCAGGCGCCATGGACTGGAAGACACTCCAG...,521,4070
4,MMACHC,chr1,+,45965724,45976739,45966004,45974887,45974467,45976739,45974647,45974648,0.454023,GGCGATGGAGGCGGTGGAACTACCTTTCTCTAGAAGAGACACGTAC...,11015,2272,8743,GGCGATGGAGGCGGTGGAACTACCTTTCTCTAGAAGAGACACGTAC...,CGCATATCAGGTGTGTGCATACACCCCCGATTTGGGGGCTGGTTTG...,180,8923
5,ACADM,chr1,+,76190035,76229364,76190472,76228448,76228376,76229364,76228405,76228406,0.327067,ACCGCGCCGCAAGTCCCCCCACCGTTCAGCGCAACCGGGCCCTCCC...,39329,988,38341,ACCGCGCCGCAAGTCCCCCCACCGTTCAGCGCAACCGGGCCCTCCC...,ATTTATGAAGGTACTTCACAAATTCAAAGACTTATTGTAGCCCGTG...,29,38370
6,FMO3,chr1,+,171060017,171086959,171061799,171086582,171086239,171086959,171086454,171086455,0.063767,GTAGGCTCACTAGAACATTTTCTCTTTCAAACTGCCCAGACGGTTG...,26942,720,26222,GTAGGCTCACTAGAACATTTTCTCTTTCAAACTGCCCAGACGGTTG...,GTTTGGCAAAAGCGAGACCATACAGACAGATTACATTGTTTATATG...,215,26437
7,ZBTB18,chr1,+,244214584,244220778,244214733,244218672,244217089,244220778,244218551,244218552,0.279094,AGTGTGTGCGGATCTGTGGTGGAGAAGGTATCTCATTCCTCTCTAA...,6194,3689,2505,AGTGTGTGCGGATCTGTGGTGGAGAAGGTATCTCATTCCTCTCTAA...,GTTATGAAGACAGTATGGAGTTTCCAGACCATAGTAGACATTTGCT...,1462,3967
8,ZBTB18,chr1,+,244214584,244220778,244214733,244218672,244217089,244220778,244218541,244218542,0.471876,AGTGTGTGCGGATCTGTGGTGGAGAAGGTATCTCATTCCTCTCTAA...,6194,3689,2505,AGTGTGTGCGGATCTGTGGTGGAGAAGGTATCTCATTCCTCTCTAA...,GTTATGAAGACAGTATGGAGTTTCCAGACCATAGTAGACATTTGCT...,1452,3957
9,MYCN,chr2,+,16080685,16087129,16082186,16086219,16085614,16087129,16086173,16086174,0.739075,ATCTGTCTGGACGCGCTGGGTGGATGCGGGGGGCTCCTGGGAACTG...,6444,1515,4929,ATCTGTCTGGACGCGCTGGGTGGATGCGGGGGGCTCCTGGGAACTG...,atgatgaagatgatgaagaggaagatgaagaggaagaaATCGACGT...,559,5488


In [12]:
PreSequence = Data["PreSequence"].tolist()
LastExonSequence = Data["LastExonSequence"].tolist()

In [13]:
for i in range(len(Data)):
    
    pre = PreSequence[i]
    last = LastExonSequence[i]
    
    Motif = last[((Data.loc[i,"m6APosition"])-2):((Data.loc[i,"m6APosition"])+3)]
    
    print(Motif)
    


AGACA


In [14]:
for i in range(len(Data)):
    
    pre = PreSequence[i]
    last = LastExonSequence[i]
    
    seq = last[((Data.loc[i,"m6APosition"])-50):((Data.loc[i,"m6APosition"])+51)]
    seq = seq.upper()
    
    print(seq)

CCGAGGTCAACAGCTTCACGGCTGCCTTCCTCTTCTCCATTGAGACCCAGACAACCATAGGCTATGGTTTCAGATGTGTCACGGATGAATGCCCAATTGCT


### Saturation Single Nucleotide Mutation

In [15]:
for i in range(len(Data)):
    
    pre = PreSequence[i]
    last = LastExonSequence[i]
    
    Pre = pre[0:(len(pre)-1)]
    LastExon = pre[(len(pre)-1):(len(pre))] + last
    
    df = pd.DataFrame(np.random.randn((len(LastExon)), 4))
    df.columns = ["A", "T", "C","G"]
    df["A"] = 0
    df["T"] = 0
    df["C"] = 0
    df["G"] = 0    
    
    for j in range(len(LastExon)):
        
        seq = LastExon[0:j] + "A" + LastExon[(j+1):]
        input_sequence = Pre + seq
        x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
        y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
        m6AAI_prob = y[0, :, 1]
        value = m6AAI_prob[(Data.loc[i,"POS"])-1]
        Dvalue = value - Data.loc[i,"ProbREF"]
        df.loc[j,"A"] = Dvalue
            
        seq = LastExon[0:j] + "T" + LastExon[(j+1):]
        input_sequence = Pre + seq
        x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
        y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
        m6AAI_prob = y[0, :, 1]
        value = m6AAI_prob[(Data.loc[i,"POS"])-1]
        Dvalue = value - Data.loc[i,"ProbREF"]        
        df.loc[j,"T"] = Dvalue

        seq = LastExon[0:j] + "C" + LastExon[(j+1):]
        input_sequence = Pre + seq
        x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
        y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
        m6AAI_prob = y[0, :, 1]
        value = m6AAI_prob[(Data.loc[i,"POS"])-1]
        Dvalue = value - Data.loc[i,"ProbREF"]        
        df.loc[j,"C"] = Dvalue
            
        seq = LastExon[0:j] + "G" + LastExon[(j+1):]
        input_sequence = Pre + seq
        x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
        y = np.mean([models[m].predict(x) for m in range(5)], axis=0)
        m6AAI_prob = y[0, :, 1]
        value = m6AAI_prob[(Data.loc[i,"POS"])-1]
        Dvalue = value - Data.loc[i,"ProbREF"]        
        df.loc[j,"G"] = Dvalue            
        
    df.to_csv("./AnalyzeMotif/{}.csv".format(Data.loc[i,"name"]+"_"+str(Data.loc[i,"m6AStart"])))



/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':
